#### Basic usage of the Bloom model for text prediction - zero-shot

##### Setup

In [ ]:
# Bloom is part of the transformers library --> install it
!pip install transformers

In [ ]:
# imports needed libraries
import torch
from transformers import AutoTokenizer,AutoModelForCausalLM # A general model for casual inferencing

In [3]:
# test GPU avialiablity - otherwise cpu

!nvidia-smi
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
device


Sun Sep 25 06:35:21 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   33C    P0    26W / 250W |      0MiB / 16280MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

device(type='cuda', index=0)

In [4]:
# if we have gpu bind all tensors to gpu, otherwise by default cpu

if 'cuda' in str(device):
  torch.set_default_tensor_type(torch.cuda.FloatTensor) # this will allocate all tensors  on cuda



In [ ]:
# define the tokenizer and model 

tokenizer = AutoTokenizer.from_pretrained("bigscience/bloom-3b")

model = AutoModelForCausalLM.from_pretrained("bigscience/bloom-3b") # here we use bloom 3b parameters

In [16]:
# define any text prompt / or a list of prompts 
# in this example we have text continuation task as well as general knowledge question/answering task mixed together
text_prompt = ["Albert Einstein won a Nobel prize for", "Who was Otto Warburg?" ]# ANyone can change this part

In [17]:
# let's look at how bloom tokenizes text
tokens = tokenizer.tokenize(text_prompt)
print(tokens)

['Albert', 'ĠEinstein', 'Ġwon', 'Ġa', 'ĠNobel', 'Ġprize', 'Ġfor', 'Who', 'Ġwas', 'ĠOtto', 'ĠWar', 'burg', '?']


In [18]:
# convert tokens to inpt ids and return pytorch tensors
input_ids = tokenizer (text_prompt, return_tensors="pt", padding=True)
input_ids

{'input_ids': tensor([[124190,  78426,  15974,    267,  41530, 127901,    613],
        [     3,  57647,   1620,  96624,  18692,  19616,     34]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1],
        [0, 1, 1, 1, 1, 1, 1]])}

In [19]:
# generate tokens as ids and review the generated ids
gen_text = model.generate(**input_ids, min_length=20, max_length=40, temperature=0.2)

In [20]:
# predict possible continuation for the provided prompts

predictions = []
for index, _ in enumerate(gen_text):
  predicted_text = tokenizer.decode(gen_text[index])
  # print(f"Paragraph {index} is: {predicted_text}\n")
  predictions.append(predicted_text)

In [21]:
# do some cleaning of text:
# 1. find the last "." and delete everything afterwords to have a clean sentense
# 2. remove the new line character \n to make it easier and review
# 3. remove <pad> tokens

for item, pred in enumerate(predictions):
  pred = pred[0:pred.rfind(".")+1] # truncate words beyond last period
  pred = pred.replace("\n", "") # remove newlines
  pred = pred.replace("<pad>", "") # remove padding tokens
  print(f"Paragraph {item}: {pred}\n")

Paragraph 0: Albert Einstein won a Nobel prize for his work on the theory of relativity. He was the first person to use the word “relativity” in his Nobel lecture.

Paragraph 1: Who was Otto Warburg?"- Otto Warburg was a German chemist who was the first to study the metabolism of cancer cells.

